In [27]:
# Script to bring the videos to my directory.

import os
import json

with open('/home/data-proyect/v1_split_trailer.json', 'r') as file:
    data = json.load(file)


def walkdir(folder):

    for dirpath, _, files in os.walk(folder):
        for filename in files:
            yield (dirpath, filename)

pos = '../data-proyect/trailer/'

os.makedirs('./data/train', exist_ok=True)
os.makedirs('./data/val', exist_ok=True)
os.makedirs('./data/test', exist_ok=True)

c_train = 0
c_test = 0
c_val = 0
none = 0


for dir,filename in walkdir(pos):
    folder = (dir[len(dir)-9:len(dir)])
    if folder in data['train']:
        c_train = c_train + 1
        src = os.path.join(pos, folder+"/"+filename)
        os.makedirs('./data/train/'+folder, exist_ok=True)
        os.link(src, os.path.join('./data/train/', folder + "/" + filename))
    elif folder in data['test']:
        c_test = c_test + 1
        src = os.path.join(pos, folder+"/"+filename)
        os.makedirs('./data/test/'+folder, exist_ok=True)
        os.link(src, os.path.join('./data/test/', folder + "/" + filename))
    elif folder in data['val']:
        c_val = c_val + 1
        src = os.path.join(pos, folder+"/"+filename)
        os.makedirs('./data/val/'+folder, exist_ok=True)
        os.link(src, os.path.join('./data/val/', folder + "/" + filename))
    else:
        pass
        
print('train: ', c_train)
print('test: ', c_test)
print('val: ', c_val)
print('none: ', none)

train:  20856
test:  8187
val:  4610
none:  0


In [2]:
# Script to generate the main dataframe

import pandas as pd
import json
import cv2
from tqdm import tqdm
import os


with open('../../../data-proyect/v1_split_trailer.json', 'r') as file:
    data = json.load(file)

dfs = pd.DataFrame()

dfs['assignment'],
dfs['video'],
dfs['scene'], 
dfs['scale_label'],
dfs['scale_value'], 
dfs['movement_label'], 
dfs['movement_value'], 
dfs['frames'], 
dfs['time(s)'], 
dfs['size(MB)'], 
dfs['dir'] = None, None, None, None, None, None, None, None, None, None, None


for j in tqdm(data, desc='Total'):
    for i in tqdm(data[j], desc='Sub-p'):
        
        for k in data[j][i]:
            
            d_scale_label = data[j][i][k]['scale']['label']
            
            d_scale_value = data[j][i][k]['scale']['value']
            
            d_movement_label = data[j][i][k]['movement']['label']
            
            d_movement_value = data[j][i][k]['movement']['value']
            
            mov = cv2.VideoCapture('../../data/' + j + '/' + i + '/shot_' + k + '.mp4')
            
            frames = mov.get(cv2.CAP_PROP_FRAME_COUNT) 
            fps = float(mov.get(cv2.CAP_PROP_FPS)) 
            seconds = f"{float(frames / fps):.02f}"
            
            length = int(mov.get(cv2.CAP_PROP_FRAME_COUNT))
            
            file_size = os.path.getsize(r'../../data/' + j + '/' + i + '/shot_' + k + '.mp4') 
            file_size = (f'{file_size/1048576:.02f}')

            dirr = '../../data/' + j + '/' + i + '/shot_' + k + '.mp4'

            datev = {'assignment':j, 
                     'video':i, 
                     'scene':k, 
                     'scale_label':d_scale_label, 
                     'scale_value':d_scale_value, 
                     'movement_label':d_movement_label, 
                     'movement_value':d_movement_value,
                     'frames':length,
                     'time(s)':seconds,
                     'size(MB)':file_size,
                     'dir':dirr,
                    }
                     
            dfs = dfs.append(datev, ignore_index=True)

dfs.to_csv('dataset_video.csv', index=False)
            

Total: 100%|██████████| 3/3 [12:48<00:00, 256.22s/it]


In [4]:
dfs

,assignment,video,scene,scale_label,scale_value,movement_label,movement_value,frames,time(s),size(MB)
0,train,tt0444850,0014,CS,1,Static,4,53,2.21,0.52
1,train,tt0444850,0015,CS,1,Static,4,45,1.88,0.63
2,train,tt0444850,0016,ECS,0,Static,4,24,1.00,0.23
3,train,tt2005173,0002,MS,2,Static,4,22,0.88,0.31
4,train,tt2005173,0014,CS,1,Static,4,22,0.88,0.16
...,...,...,...,...,...,...,...,...,...,...
33648,test,tt6644200,0011,FS,3,Static,4,26,1.08,0.23
33649,test,tt6644200,0015,MS,2,Motion,0,38,1.58,0.37
33650,test,tt6644200,0021,FS,3,Motion,0,47,1.96,0.28
33651,test,tt6644200,0026,LS,4,Static,4,28,1.17,0.84
